# 注意力機制與Transformer模型

循環神經網路（RNN）的一個主要缺點是序列中的所有詞對結果的影響是相同的。這導致標準的LSTM編碼器-解碼器模型在處理序列到序列任務（如命名實體識別和機器翻譯）時表現不佳。實際上，輸入序列中的某些特定詞往往對輸出序列的影響更大。

考慮一個序列到序列的模型，例如機器翻譯。這種模型由兩個循環神經網路實現，其中一個網路（**編碼器**）將輸入序列壓縮為隱藏狀態，另一個網路（**解碼器**）將該隱藏狀態展開為翻譯結果。這種方法的問題在於，網路的最終狀態很難記住句子的開頭部分，從而導致模型在處理長句子時的質量下降。

**注意力機制**提供了一種方法，能夠對每個輸入向量對RNN每個輸出預測的上下文影響進行加權。其實現方式是通過在輸入RNN的中間狀態和輸出RNN之間創建捷徑。這樣，在生成輸出符號 $y_t$ 時，我們會考慮所有輸入隱藏狀態 $h_i$，並賦予不同的權重係數 $\alpha_{t,i}$。

![顯示具有加性注意力層的編碼器/解碼器模型的圖片](../../../../../translated_images/zh-MO/encoder-decoder-attention.7a726296894fb567.webp)
*來自 [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) 的加性注意力機制編碼器-解碼器模型，圖片引用自[這篇部落格文章](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

注意力矩陣 $\{\alpha_{i,j}\}$ 表示某些輸入詞在生成輸出序列中特定詞時所起的作用程度。以下是這樣一個矩陣的示例：

![顯示由 RNNsearch-50 找到的樣本對齊的圖片，取自 Bahdanau - arviz.org](../../../../../translated_images/zh-MO/bahdanau-fig3.09ba2d37f202a6af.webp)

*圖片取自 [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf)（圖3）*

注意力機制是當前或接近當前自然語言處理技術水平的關鍵因素。然而，添加注意力機制會大幅增加模型參數的數量，這導致了RNN的擴展問題。RNN擴展的一個主要限制是其循環特性使得訓練過程難以批量化和並行化。在RNN中，序列的每個元素都需要按順序處理，這意味著它無法輕易並行化。

注意力機制的採用結合了這一限制，促成了如今我們所熟知和使用的最先進的Transformer模型的誕生，從BERT到OpenGPT3。

## Transformer模型

與將每次預測的上下文傳遞到下一步評估不同，**Transformer模型**使用**位置編碼**和注意力機制來捕捉給定輸入在提供的文本窗口內的上下文。下圖展示了如何通過位置編碼和注意力機制在給定窗口內捕捉上下文。

![顯示Transformer模型中如何進行評估的動畫GIF](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

由於每個輸入位置可以獨立映射到每個輸出位置，Transformer模型比RNN更容易並行化，這使得構建更大且更具表達力的語言模型成為可能。每個注意力頭可以用來學習詞與詞之間的不同關係，從而改進下游的自然語言處理任務。

**BERT**（Bidirectional Encoder Representations from Transformers，雙向編碼器表示）是一個非常大的多層Transformer網路，*BERT-base*有12層，*BERT-large*有24層。該模型首先在大規模文本數據（維基百科+書籍）上進行無監督訓練（預測句子中的被遮蔽詞）。在預訓練過程中，模型吸收了大量的語言理解能力，這些能力可以通過微調其他數據集來加以利用。這個過程被稱為**遷移學習**。

![圖片來自 http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/zh-MO/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

Transformer架構有許多變體，包括BERT、DistilBERT、BigBird、OpenGPT3等，這些模型都可以進行微調。[HuggingFace套件](https://github.com/huggingface/) 提供了用PyTorch訓練這些架構的資源庫。

## 使用BERT進行文本分類

讓我們看看如何使用預訓練的BERT模型來解決我們的傳統任務：序列分類。我們將對原始的AG News數據集進行分類。

首先，讓我們加載HuggingFace庫和我們的數據集：


In [10]:
import torch
import torchtext
from torchnlp import *
import transformers
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_len = len(vocab)

Loading dataset...
Building vocab...


因為我們將使用預訓練的 BERT 模型，所以需要使用特定的分詞器。首先，我們會載入與預訓練 BERT 模型相關聯的分詞器。

HuggingFace 庫包含一個預訓練模型的存儲庫，您只需在 `from_pretrained` 函數中指定模型名稱作為參數即可使用。所有模型所需的二進制文件都會自動下載。

然而，有時您可能需要載入自己的模型，在這種情況下，您可以指定包含所有相關文件的目錄，包括分詞器的參數、模型參數的 `config.json` 文件、二進制權重等。


In [11]:
# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

`tokenizer` 物件包含 `encode` 函數，可直接用於編碼文本：


In [15]:
tokenizer.encode('PyTorch is a great framework for NLP')

[101, 1052, 22123, 2953, 2818, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

然後，讓我們建立在訓練期間用於訪問數據的迭代器。由於 BERT 使用其自己的編碼函數，我們需要定義一個類似於之前定義的 `padify` 的填充函數：


In [4]:
def pad_bert(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [tokenizer.encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0] for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, collate_fn=pad_bert, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, collate_fn=pad_bert)

在我們的案例中，我們將使用名為 `bert-base-uncased` 的預訓練 BERT 模型。讓我們使用 `BertForSequenceClassfication` 套件來加載模型。這確保了我們的模型已經具備分類所需的架構，包括最終的分類器。您會看到一條警告消息，指出最終分類器的權重尚未初始化，並且模型需要進行預訓練——這完全沒問題，因為這正是我們即將進行的操作！


In [9]:
model = transformers.BertForSequenceClassification.from_pretrained(bert_model,num_labels=4).to(device)

Some weights of the model checkpoint at ./bert were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert and

現在我們準備開始訓練了！由於 BERT 已經過預訓練，我們希望以較小的學習率開始，這樣可以避免破壞初始權重。

所有的繁重工作都由 `BertForSequenceClassification` 模型完成。當我們將訓練數據傳入模型時，它會返回該批次的損失值和網絡輸出。我們使用損失值進行參數優化（`loss.backward()` 負責反向傳播），而使用 `out` 計算訓練準確率，方法是將獲得的標籤 `labs`（通過 `argmax` 計算）與預期的 `labels` 進行比較。

為了控制訓練過程，我們會在多次迭代中累積損失值和準確率，並在每 `report_freq` 個訓練週期後打印出來。

這次訓練可能會花費相當長的時間，因此我們會限制迭代次數。


In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

report_freq = 50
iterations = 500 # make this larger to train for longer time!

model.train()

i,c = 0,0
acc_loss = 0
acc_acc = 0

for labels,texts in train_loader:
    labels = labels.to(device)-1 # get labels in the range 0-3         
    texts = texts.to(device)
    loss, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc = torch.mean((labs==labels).type(torch.float32))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    acc_loss += loss
    acc_acc += acc
    i+=1
    c+=1
    if i%report_freq==0:
        print(f"Loss = {acc_loss.item()/c}, Accuracy = {acc_acc.item()/c}")
        c = 0
        acc_loss = 0
        acc_acc = 0
    iterations-=1
    if not iterations:
        break

Loss = 1.1254194641113282, Accuracy = 0.585
Loss = 0.6194715118408203, Accuracy = 0.83
Loss = 0.46665248870849607, Accuracy = 0.8475
Loss = 0.4309701919555664, Accuracy = 0.8575
Loss = 0.35427074432373046, Accuracy = 0.8825
Loss = 0.3306886291503906, Accuracy = 0.8975
Loss = 0.30340143203735354, Accuracy = 0.8975
Loss = 0.26139299392700194, Accuracy = 0.915
Loss = 0.26708646774291994, Accuracy = 0.9225
Loss = 0.3667240524291992, Accuracy = 0.8675


你可以看到（尤其是當你增加迭代次數並等待足夠長的時間），BERT 分類能夠提供相當不錯的準確率！這是因為 BERT 已經對語言的結構有相當好的理解，我們只需要微調最終的分類器即可。然而，由於 BERT 是一個大型模型，整個訓練過程需要很長的時間，並且需要強大的計算能力！（GPU，最好是多個）。

> **Note:** 在我們的範例中，我們使用的是最小的預訓練 BERT 模型之一。還有更大的模型可能會產生更好的結果。


## 評估模型表現

現在我們可以在測試數據集上評估模型的表現。評估循環與訓練循環非常相似，但我們不能忘記通過呼叫 `model.eval()` 將模型切換到評估模式。


In [10]:
model.eval()
iterations = 100
acc = 0
i = 0
for labels,texts in test_loader:
    labels = labels.to(device)-1      
    texts = texts.to(device)
    _, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc += torch.mean((labs==labels).type(torch.float32))
    i+=1
    if i>iterations: break
        
print(f"Final accuracy: {acc.item()/i}")

Final accuracy: 0.9047029702970297


## 重點

在本單元中，我們已經看到如何輕鬆地從 **transformers** 庫中取用預訓練的語言模型，並將其調整為我們的文本分類任務。同樣地，BERT 模型也可以用於實體抽取、問題回答以及其他 NLP 任務。

Transformer 模型代表了 NLP 領域的最新技術，在大多數情況下，當你開始實現自定義 NLP 解決方案時，它應該是你首先嘗試的選擇。然而，如果你希望構建更高級的神經模型，理解本模組中討論的循環神經網絡的基本原理是非常重要的。



---

**免責聲明**：  
本文件已使用 AI 翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 進行翻譯。儘管我們努力確保翻譯的準確性，但請注意，自動翻譯可能包含錯誤或不準確之處。原始文件的母語版本應被視為權威來源。對於關鍵資訊，建議使用專業人工翻譯。我們對因使用此翻譯而引起的任何誤解或錯誤解釋不承擔責任。
